<a href="https://colab.research.google.com/github/GianmarcoAndreana/NNDL/blob/main/Perceptron_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################
#wget file
####################
!wget https://archive.ics.uci.edu/static/public/17/breast+cancer+wisconsin+diagnostic.zip
!unzip breast+cancer+wisconsin+diagnostic.zip
!ls
#!cat wdbc.data
#!cat wdbc.names

In [ ]:
####################
#import to pandas
####################
import pandas as pd

# Load the dataset to inspect its contents
file_path = 'wdbc.data'
data = pd.read_csv(file_path, header=None)

# Assign column names based on the Breast Cancer Wisconsin dataset documentation
columns = ['ID', 'Diagnosis'] + [f'Feature_{i}' for i in range(1, 31)]
data.columns = columns

# Display the first few rows to understand the structure
data.head()

In [ ]:
####################
#convert data to numpy
####################
import numpy as np

# Convert the diagnosis column into numerical values for plotting (M = 1, B = -1)
data['Diagnosis'] = data['Diagnosis'].map({'M': 1, 'B': -1})

# Extract features&labels from pandas df and convert to numpy arrays (.value)
X = data[['Feature_1', 'Feature_2']].values
y = data['Diagnosis'].values

#NOTE: X is a 569 x 2 matrix: a row per sample, a column per feature

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the Perceptron Model
class Perceptron(nn.Module):
    def __init__(self, input_dim):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.tanh(self.linear(x))  # Activation function

# # Standardize the data (important for correct decision boundary)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Initialize model, loss, and optimizer
model = Perceptron(input_dim=2)
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

# Train perceptron
for epoch in range(50):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optimizer.step()

# Extract learned weights and bias
w = model.linear.weight.data.numpy().flatten()
b = model.linear.bias.data.numpy()

# Plot the data
plt.figure(figsize=(8, 6))
# plt.scatter(X[y[:, 0] == 1][:, 0], X[y[:, 0] == 1][:, 1],  edgecolor="k", alpha=0.6, marker="o", label="Malign (+1)", s=100)
# plt.scatter(X[y[:, 0] == -1][:, 0], X[y[:, 0] == -1][:, 1], edgecolor="k", alpha=0.6, marker="o", label="Benign (-1)", s=100)
plt.scatter(
    X[:, 0], X[:, 1], c=y, cmap="coolwarm", edgecolor="k", alpha=0.8, marker="o", s=100,
)
# Plot decision boundary
x_boundary = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
y_boundary = (-w[0] * x_boundary - b) / w[1]  # Compute y values

# Legend for the plot
handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', alpha = 0.8, markersize=8, label='Benign (-1)'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', alpha = 0.8, markersize=8, label='Malignant (1)')
]
plt.legend(handles=handles, title='Diagnosis')

plt.plot(x_boundary, y_boundary, 'k--', lw=3, label="Decision Boundary")
plt.xlabel("Radius Mean (scaled)")
plt.ylabel("Texture Mean (scaled)")
plt.title("Perceptron as NN")
plt.grid(True)
plt.show()